# LSTM
<font size=5 face='楷体'>
RNN的结构决定了它的输入元素的记忆与距离呈正相关,即RNN不能很好地处理长期依赖问题．RNN结构本质上是很多层相同非线性函数的嵌套，若忽略激活函数和输入向量，状态$h_t$可以表示成:$h_t=\boldsymbol{W}h_{t_1}=\boldsymbol{W}^2h_{t-1}=...=\boldsymbol{W}^th_0$,若对参数矩阵$\boldsymbol{W}$进行特征分解,$\boldsymbol{W=Q\Lambda Q^{-1}}$,则状态节点$h_t$可表示为$h_t=\boldsymbol{Q^{-1}\Lambda^tQ}h_0$,若特征值小于1,则$h_t$特征值会快速降为0,若特征值大于1,则趋于爆炸.<br/>
长短记忆网络(Long Short Term Memory Network,LSTM)是循环神经网络的改进,Sepp Hochreiter和Jurgen Schmidhuber于1997年提出.主要改进是多出了三个门控制器:输入门(input gate),输出门(output gate)和遗忘门(forget gate).三个门控制器的结构都相同,主要由sigmoid函数和点积操作构成,由于sigmoid函数的取值范围为[0,1],所以门控制器描述了信息能够通过的比例,sigmoid函数取值为0的时候表示没有信息通过,或者理解为所有记忆都遗忘,取1则分支所有记忆都保留.
LSTM模型在所有短期记忆单元$h_t$的基础上,增加一个记忆单元$C_t$来保持长期记忆.<br/>
</font>

## 长记忆单元
<font size=5 face='楷体'>
长记忆单元,公式表示为:$$C_t=f_t\times C_t+i_t\times \tilde{C_t}$$  $$f_t=\sigma (W_t\cdot [h_{t-1},x_t]+b_f)$$  $$i_t=\sigma (W_i\cdot [h_{t-1},x_t]+b_i)$$  $$\tilde{C_t}=tanh(W_c\cdot [h_{t-1},x_t]+b_C)$$
其中,$f_t,i_t$分别代表遗忘门和输入门,在每一时刻,遗忘门会控制上一时刻记忆的遗忘程度,输入门会控制新记忆$\tilde{C_t}$的写入长期记忆的程度.$f_t,i_t$和$\tilde{C_t}$都是与上一时刻的短记忆$h_{t-1}$和当前时刻输入$x_t$相关的函数.并且$f_t$和$i_t$是sigmold函数,所以取值范围为[0,1],$\tilde{C_t}$为tanh函数,取值范围为[-1,1].
 </font>

## 短记忆单元
<font size=5 face='楷体'>
短记忆单位,公式表示为:$$h_t=o_t\times tanh(C_t)$$  $$o_t=\sigma(W_o*[h_{t-1},x_t]+b_o)$$
其中,$o_t$表示输出门,它控制着短期记忆如何受长期记忆影响.
    </font>

## 主要特点
<font size=5 face='楷体'>
LSTM对标准RNN的改进主要体现在通过门控制器增加了对不同时刻记忆的权重控制,以及加入跨层连接削减梯度消失问题的影响.LSTM实际上在原有结构上增加了线性连接,而不再是单纯的非线性连接叠加,这样能使长期信息更好地传播,这与深度残差网络ResNet通过增加跨层连接来消除梯度消失问题的影响在设计上有异曲同工之妙.<br/>

LSTM通过门控制器和新的记忆单元,在RNN原有的短期记忆之上保留了长期记忆.如果一个事件非常重要,则输入门就按重要程度将短期记忆合并进长期记忆,或者通过遗忘门忘记部分长期记忆,按比例替换为现在的新记忆,而在最后,输出门会基于长期记忆和短期记忆综合判断到底该有什么样的输出. 
</font>

In [ ]:
![avatar](https://raw.githubusercontent.com/hzg0601/python/master/LSTM.jpg)
jupyter nbconvert telematics.ipynb --to slides --post serve


<img src="https://raw.githubusercontent.com/hzg0601/python/master/LSTM.jpg" width="800" height="700"/>

# LSTM-Telematics
<font size=5 face='楷体'>
对于每个司机，共选取200个里程，其中10-50个来自其他司机作为负样例，其余来自该司机作为正样例，将里程匹配问题作为一个二分类问题。<br/>
每四秒截取一次行驶轨迹作为单位里程窗口，采集最大、最小、平均的速度、加速度、转弯角度共15个（速度$\times2\times3$,加速度$\times2\times3$,转弯角度$\times1\times3$）作为特征向量，因此对于超过200秒的里程即有50个特征向量，为便于处理,所有里程修正为200秒的最小倍数，将里程序列修正为50的倍数。<br/>
    经过处理后，每个里程被变换为以一个长度为15的向量为元素的<font color='red'>变长序列</font>，每50个特征向量作为一个batch,同时修正对应标签。因此可以利用seq2seq的方式进行建模。其中BPTT算法的时长为50，以softmax进行预测，cross-entropy为目标函数，正则化的方法为dropout(时长和隐层两个维度上)，优化方法为Adagrad，使用每5个epochs下降一次的学习率模拟退火策略<br/>
</font>

# 里程匹配
## kaggle
<font size=5 face='楷体'>
    **同时考虑分段里程的距离、转弯角度、方向，将匹配得分与负样例比较得到的匹配得分相比较，计算相应的概率。除了里程匹配，还有其他特征包括速度，加速度，停车前速度等等。**<br/><br/>
    **首先在里程匹配阶段计算每个里程匹配的路线；对所有匹配到两个或以上里程的路线，以它们匹配到的其他里程排序，再按telematic的结果排序，得的约40万个里程；对于另外14万7千个里程，它们是否能够影响telematic基于它们是否存在里程匹配：如果有，调高telematic得分；如果是长里程，并且没有里程匹配，调低在telematic的结果。**<br/><br/>
    **首先进行RDP简化，针对某个司机的所有路程，获得本次里程与其他不属于该司机的里程的最大连续匹配。**<br/>
    </font>
    
    
    

<font size=5 face='楷体'>
    **利用RDP算法对里程进行简化，定义一个转弯的阈值，以及最小转弯次数，然后将超过阈值的转弯所分割出的距离合并成一个列表，同时再建一张表记录转弯的大小。根据所分割的距离，和转弯角度去进行匹配。（标准化后的均方误？）**<br/>
    **一个里程与其他里程连续5次转弯的匹配次数（转弯间的距离与转弯角度），以RSME计算子里程的匹配次数作为额外的特征。post train boost(把匹配结果\*1.1)**<br/>
    ***用RDP进行简化，将每次转弯间的路程和转弯的角度记录下来，增加稍大或稍小角度的冗余，前向或后向进行匹配，利用SVD进行对齐,比较对齐的结果,记录每个里程匹配的个数，然后将这些特征进行机器学习。***<br/>
    基于转弯的坐标对里程进行分段匹配并对齐，然后进行SVD，然后将分解的结果对齐子里程，计算所有点的RSME <br/>
    用RDP对路线进行简化，对齐并并进行SVD分解，计算子里程的RSME，然后将整个里程对齐，进行而获得整个路程从开始到结束的距离测度。<br/>
    </font>
    
    


<font size=5 face='楷体'>
**首先进行考虑转弯的修正RDP（确定起始点）简化，加入速度、加速度等特征提升算法。**<br/><br/>
    **对每个里程，创建一个包含每x米瞬时转向的向量（以不同的滞后去捕捉50/100米停车的里程）；然后根据累计角度的查分的方差为里程的相似性进行打分；将一组里程与每个司机的每个里程比较，在这个阶段会得的不匹配的里程得分；最后对于每个司机，去计算200个里程得分，并与以获得的里程分布比较，并修正概率。**<br/>
    </font>

# 度量里程相似度
## 仅路线
<font size=5 face='楷体'>
    1,利用RDP算法对所有里程进行简化。<br/>
    2，提取所有里程分段的距离、角度（包含方向右转为正，左转为负）.<br/>
    3，计算距离、角度、转向的rmse，记为$r1_i,r2_i$。<br/>
    4，对于另一其他里程j，以三者比值的乘积作为相似度指标$\sigma_{ij}=\frac{r1_i}{r1_j}\times\frac{r2_i}{r2_j}$
    
    

## 包括驾驶信息
<font size=5 face='楷体'>
    1,利用RDP算法对所有里程进行简化。<br/>
    2，提取所有里程分段的距离、角度、速度、加速度、停车前速度.<br/>
    3，计算距离、角度、速度、加速度、停车前速度的rmse，记为$r1_i,r2_i,r3_i，r4_i,r5_i$。<br/>
    4，对于另一其他里程j，以三者比值的乘积作为相似度指标$\sigma_{ij}=\prod_{n=1}^5{\frac{rn_i}{rn_j}}$
    注：用于任意两个路线的相似度度量,若用于单个里程得分可用$\sigma_{ij}=\prod_{n=1}^5{rn_i}$；此外，若考虑指标的方差可能为0，可将公式修正为$\sigma_{ij}=\prod_{n=1}^5{（rn_i+1）}$


## 基于序列建模
<font size=5 face='楷体'>
按照固定的时间窗(或者距离)采集速度、加速度、转弯角度生成变长序列，进行序列建模（RNN\LSTM\GRU)，按照出险次数进行多分类任务建模。</font>

## 基于CNN方法
<font size=5 face='楷体'>
对于每个驾驶人，采集相同的数量(n)的里程，利用RDP算法对每次里程进行修正，修正为相同数量转弯点（m）的里程，采集速度、加速度、转弯角度共p个特征，
构成一个$n\times m\times p$的“图片”，利用CNN方法进行多分类任务建模。</font>

## 基于集成的方法
<font size=5 face='楷体'>
    采集一段时间内出现次数、每次出行的距离、转弯次数、停车时间作为出行规律程度的测度指标，采集一段时间内出现超速、急加速、急减速、急转弯等指标作为驾驶规律程度的测度指标，通过GBDT、Adaboost、Random Forest、Stacking等方法进行多分类任务建模。</font>
    

<img src='https://raw.githubusercontent.com/hzg0601/python/master/%E6%9C%AA%E5%91%BD%E5%90%8D%E6%96%87%E4%BB%B6.png' >